# Predicción con regresión logistica

## Alerta deriesgo de nuevos clientes

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import pickle


# carga de datos

df = pd.read_csv("clientes_credito.csv")

# Features base

X = df[["Ingresos_Anuales_USD", "Deuda_Total_USD"]].copy()

# Feature extra: ratio deuda/ingreso

X["Ratio_Deuda_Ingreso"] = X["Deuda_Total_USD"] / X["Ingresos_Anuales_USD"]

y = df["Riesgo_Crediticio"]

# Split estratificado para mantener proporciones de clases

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)


# pipeline: scaler + logreg

model = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        multi_class="multinomial",
        solver="lbfgs",
        max_iter=2000,
        class_weight="balanced",  # útil si alguna clase queda con pocos casos
        random_state=42
    ))
])

model.fit(X_train, y_train)


# Evaluación rápida

pred_test = model.predict(X_test)
print("Confusion matrix:\n", confusion_matrix(y_test, pred_test))
print("\nReport:\n", classification_report(y_test, pred_test))

# Guardar modelo

with open("modelo_riesgo_crediticio.pkl", "wb") as f:
    pickle.dump(model, f)

# Predicción sobre nuevos clientes

df_nuevos = pd.read_csv("nuevos_clientes.csv")

X_new = df_nuevos[["Ingresos_Anuales_USD", "Deuda_Total_USD"]].copy()
X_new["Ratio_Deuda_Ingreso"] = X_new["Deuda_Total_USD"] / X_new["Ingresos_Anuales_USD"]

proba = model.predict_proba(X_new)
clases = model.named_steps["clf"].classes_

# salida

df_proba = pd.DataFrame(proba, columns=[f"Proba_{c}" for c in clases])
df_out = pd.concat([df_nuevos, df_proba], axis=1)

df_out["Riesgo_Predicho"] = clases[np.argmax(proba, axis=1)]

# Umbral de alerta: si P(malo) > 0.35 => alertar

umbral_alerta = 0.35
col_malo = "Proba_malo"
df_out["Alerta"] = np.where(df_out[col_malo] > umbral_alerta, "SI", "NO")

# Ordenar para revisar rápido (primero los más riesgosos)

df_out = df_out.sort_values(by=col_malo, ascending=False).reset_index(drop=True)

print("\nPredicciones (top):")
print(df_out[["ID", "Nombre", "Ingresos_Anuales_USD", "Deuda_Total_USD", "Riesgo_Predicho", col_malo, "Alerta"]])

# guardar el resultado:

df_out.to_csv("predicciones_nuevos_clientes.csv", index=False)
print("\nArchivo generado: predicciones_nuevos_clientes.csv")


Confusion matrix:
 [[1 0 0]
 [0 4 0]
 [0 0 2]]

Report:
               precision    recall  f1-score   support

       bueno       1.00      1.00      1.00         1
        malo       1.00      1.00      1.00         4
     regular       1.00      1.00      1.00         2

    accuracy                           1.00         7
   macro avg       1.00      1.00      1.00         7
weighted avg       1.00      1.00      1.00         7


Predicciones (top):
     ID   Nombre  Ingresos_Anuales_USD  Deuda_Total_USD Riesgo_Predicho  \
0  1006  Facundo                 81991            82136            malo   
1  1012  Leandro                 92757            87668            malo   
2  1010  Joaquin                 11664            13603            malo   
3  1017    Rocio                 50216            41955            malo   
4  1014  Nicolas                 73024            42845            malo   
5  1002    Bruno                 74203            41548            malo   
6  1022   Walter

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
